In [ ]:
# @title Copyright & License (click to expand)
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Monitoring for AutoML tabular models

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring/get_started_with_model_monitoring_automl.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fmodel_monitoring%2Fget_started_with_model_monitoring_automl.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring/get_started_with_model_monitoring_automl.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br>
      View on GitHub
    </a>
  </td> 
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/model_monitoring/get_started_with_model_monitoring_automl.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br>
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Overview

This tutorial demonstrates how to use Vertex AI Model Monitoring for AutoML tabular models.

Learn more about [Vertex AI Model Monitoring](https://cloud.google.com/vertex-ai/docs/model-monitoring).

### Objective

In this notebook, you learn to use the Vertex AI Model Monitoring service to detect feature skew and drift in the input predict requests, for AutoML tabular models.

This tutorial uses the following Vertex AI services:

- AutoML
- Vertex AI Model Monitoring
- Vertex AI Prediction
- Vertex AI model resource
- Vertex AI endpoint resource

The steps performed include:

- Train an AutoML model.
- Deploy the model resource to a Vertex AI endpoint resource.
- Configure the endpoint resource for model monitoring.
- Generate synthetic prediction requests for skew.
- Generate synthetic prediction requests for drift.
- Wait for email alert notification.

Learn more about [Introduction to Vertex AI Model Monitoring](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview).

### Dataset

The dataset used for this tutorial is the GSOD dataset from [BigQuery public datasets](https://cloud.google.com/bigquery/public-data). In this notebook, you use only the fields year, month and day from the dataset to predict the value of mean daily temperature (mean_temp).

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* BigQuery
* Cloud Storage

Learn about [Vertext AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Get started

### Install Vertex AI SDK for Python and other required packages

In [ ]:
# Install required packages.
! pip3 install  --quiet --upgrade google-cloud-aiplatform \
                                  google-cloud-bigquery

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"

### User Email

Set your user email address to receive monitoring alerts.

In [ ]:
import os

USER_EMAIL = "[your-email-address]"  # @param {type:"string"}

if os.getenv("IS_TESTING"):
    USER_EMAIL = "noreply@google.com"

### Notes about service account and permission

**By default no configuration is required**, if you run into any permission related issue, make sure the service accounts above have the required roles:

|Service account email|Description|Roles|
|---|---|---|
|PROJECT_NUMBER-compute@developer.gserviceaccount.com|Compute Engine default service account|Dataflow Admin, Dataflow Worker, Storage Admin, BigQuery Admin, Vertex AI User|
|service-PROJECT_NUMBER@gcp-sa-aiplatform.iam.gserviceaccount.com|AI Platform Service Agent|Vertex AI Service Agent|


1. Go to the [IAM console](https://console.cloud.google.com/iam-admin/iam).
2. Check the **Include Google-provided role grants** checkbox.
3. Find the above emails.
4. Grant the corresponding roles.

### Using data source from a different project
If you're using data sources from a different project:
- For BigQuery data source, grant the "BigQuery Data Viewer" role to both the service accounts.
- For CSV data source, grant the "Storage Object Viewer" role to both the service accounts.

### Import libraries 

In [ ]:
import os

import google.cloud.aiplatform as aiplatform
from google.cloud import bigquery
from google.cloud.aiplatform import model_monitoring

### Initialize Vertex AI SDK for Python

To get started using Vertex AI, you must [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION)

### Create BigQuery client

In this tutorial, you use data from the public BigQuery table. You create a client interface, which you then use to access the data.

In [ ]:
bqclient = bigquery.Client(project=PROJECT_ID)

## Introduction to Vertex AI Model Monitoring

Vertex AI Model Monitoring is supported for AutoML tabular models and custom tabular models. You can monitor for skewness and drift detection of the features in the inbound prediction requests or the feature attributions (Explainable AI) in the outbound prediction response. That is, you monitor the distribution of the attributions that quantify feature contributions to the output (predictions).

The following are the basic steps to enable model monitoring:

1. Deploy a Vertex AI AutoML or custom tabular model to a Vertex AI endpoint.
2. Configure a model monitoring specification.
3. Upload the model monitoring specification to the Vertex AI endpoint.
4. Upload schema or use automatic generation of the *input schema* for parsing.
5. For feature skewness detection, upload the training data. This enables automatic generation of the feature distributions.
6. For feature attributions, upload the corresponding *Vertex Explainable AI* specification.

Once configured, you can enable or disable monitoring, change alerts and update the model monitoring configuration. 

When model monitoring is enabled, the sampled incoming prediction requests are logged into a BigQuery table. The input feature values contained in the logged requests are then analyzed for skewness or drift on an specified interval basis. You set a sampling rate to monitor a subset of the production inputs to the model, and the monitoring interval.

The model monitoring service needs to know how to parse the feature values, which is referred to as the input schema. For AutoML tabular models, the input schema is automatically generated. For custom tabular models, the service attempts to automatically derive the input schema from the first 1000 prediction requests. Alternatively, one can upload the input schema.

For skewness detection, the monitoring service requires a baseline for the statistical distribution of values in the training data. For AutoML tabular models this is automatically derived. For custom tabular models, you upload the training data to the service, and have the service automatically derive the distribution.

For feature attribution skewness and drift detection in feature attributions, you're required to enable the Vertex Explainable AI feature for your deployed custom tabular models. For AutoML models, Vertex Explainable AI is automatically enabled.

Learn more about [Introduction to Vertex AI Model Monitoring](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview).

#### Location of BigQuery training data.

Now set the variable `IMPORT_FILE` to the location of the data table in BigQuery.

In [ ]:
IMPORT_FILE = "bq://bigquery-public-data.samples.gsod"
BQ_TABLE = "bigquery-public-data.samples.gsod"

### Create the Dataset

#### BigQuery input data

Next, create the dataset resource using the `create` method for the `TabularDataset` class, which takes the following parameters:

- `display_name`: The human readable name for the dataset resource.
- `bq_source`: Import data items from a BigQuery table into the dataset resource.
- `labels`: User defined metadata. In this example, you store the location of the Cloud Storage bucket containing the user defined data.

Learn more about [TabularDataset from BigQuery table](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api#aiplatform_create_dataset_tabular_bigquery_sample-python).

In [ ]:
dataset = aiplatform.TabularDataset.create(display_name="gsod", bq_source=[IMPORT_FILE])

label_column = "mean_temp"

print(dataset.resource_name)

### Create and run training pipeline

To train an AutoML model, you perform two steps: 1) create a training pipeline, and 2) run the pipeline.

#### Create training pipeline

An AutoML training pipeline is created with the `AutoMLTabularTrainingJob` class, with the following parameters:

- `display_name`: The human readable name for the training job resource.
- `optimization_prediction_type`: The type task to train the model for.
  - `classification`: A tabuar classification model.
  - `regression`: A tabular regression model.
- `column_transformations`: (Optional): Transformations to apply to the input columns. In this example, you set the column transformations to use the default transformation based on their data type.
- `optimization_objective`: The optimization objective to minimize or maximize.
  - binary classification:
    - `minimize-log-loss`
    - `maximize-au-roc`
    - `maximize-au-prc`
    - `maximize-precision-at-recall`
    - `maximize-recall-at-precision`
  - multi-class classification:
    - `minimize-log-loss`
  - regression:
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

In [ ]:
TRANSFORMATIONS = [
    {"auto": {"column_name": "year"}},
    {"auto": {"column_name": "month"}},
    {"auto": {"column_name": "day"}},
]

label_column = "mean_temp"

In [ ]:
dag = aiplatform.AutoMLTabularTrainingJob(
    display_name="gsod",
    optimization_prediction_type="regression",
    optimization_objective="minimize-rmse",
    column_transformations=TRANSFORMATIONS,
)

print(dag)

#### Run the training pipeline

Next, you run the created DAG to start the training job by invoking the method `run`, with the following parameters:

- `dataset`: The dataset resource to train the model.
- `model_display_name`: The human readable name for the trained model.
- `training_fraction_split`: The percentage of the dataset to use for training.
- `test_fraction_split`: The percentage of the dataset to use for test (holdout data).
- `validation_fraction_split`: The percentage of the dataset to use for validation.
- `target_column`: The name of the column to train as the label.
- `budget_milli_node_hours`: (optional) Maximum training time specified in unit of millihours (1000 = hour).
- `disable_early_stopping`: If `False`, training may be completed before using the entire budget if the service detects that it can't further improve the model objective measurements.

The `run` method when completed returns the model resource.

The execution of the training pipeline can take upto 30 minutes or more.

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="gsod",
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
    target_column=label_column,
)

## Deploy the model

Next, deploy your model for online prediction. To deploy the model, you invoke the `deploy` method, with the following parameters:

- `machine_type`: The type of compute machine.

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

### Configure the alerting specification

First, you configure the `alerting_config` specification with the following settings:

- `user_emails`: A list of one or more email to send alerts to.
- `enable_logging`: Streams detected anomalies to Cloud Logging. Default is False.

In [ ]:
# Create alerting configuration.
alerting_config = model_monitoring.EmailAlertConfig(
    user_emails=[USER_EMAIL], enable_logging=True
)

### Configure the monitoring interval specification

Next, you configure the `schedule_config` specification with the following settings:

- `monitor_interval`:  Sets the model monitoring job scheduling interval in hours. Minimum time interval is 1 hour.

In [ ]:
# Monitoring Interval
MONITOR_INTERVAL = 1  # @param {type:"number"}

# Create schedule configuration
schedule_config = model_monitoring.ScheduleConfig(monitor_interval=MONITOR_INTERVAL)

### Configure the sampling specification

Next, you configure the `logging_sampling_strategy` specification with the following settings:

- `sample_rate`: The rate as a percentage (between 0 and 1) to randomly sample prediction requests for monitoring. Selected samples are logged to a BigQuery table.

In [ ]:
# Sampling rate (optional, default=.8)
SAMPLE_RATE = 0.5  # @param {type:"number"}

# Create sampling configuration
logging_sampling_strategy = model_monitoring.RandomSampleConfig(sample_rate=SAMPLE_RATE)

### Configure the drift detection specification

Next, you configure the `drift_config` specification with the following settings:

- `drift_thresholds`: A dictionary of key/value pairs where the keys are the input features for monitor for drift. The value is the detection threshold. When not specified, the default drift threshold for a feature is 0.3 (30%).

**Note:** Enabling drift detection is optional.

In [ ]:
DRIFT_THRESHOLD_VALUE = 0.05

DRIFT_THRESHOLDS = {"year": DRIFT_THRESHOLD_VALUE, "motnth": DRIFT_THRESHOLD_VALUE}

drift_config = model_monitoring.DriftDetectionConfig(drift_thresholds=DRIFT_THRESHOLDS)

### Configure the skew detection specification

Next, you configure the `skew_config` specification with the following settings:

- `data_source`: The source of the dataset of the original training data. The format of the source defaults to a BigQuery table. Otherwise the setting `data_format` must be set to one of the values below. The location of the data must be a Cloud Storage location.
  - `csv`
  - `jsonl`
  - `tf-record`
- `skew_thresholds`: A dictionary of key/value pairs where the keys are the input features for monitor for skew. The value is the detection threshold. When not specified, the default skew threshold for a feature is 0.3 (30%).
- `target_field`: The target label for the training dataset

**Note:** Enabling skewness detection is optional.

In [ ]:
# URI to training dataset.
DATASET_BQ_URI = "bq://" + BQ_TABLE
# Prediction target column name in training dataset.
TARGET = label_column

SKEW_THRESHOLD_VALUE = 0.5

SKEW_THRESHOLDS = {
    "year": SKEW_THRESHOLD_VALUE,
    "month": SKEW_THRESHOLD_VALUE,
}

skew_config = model_monitoring.SkewDetectionConfig(
    data_source=DATASET_BQ_URI, skew_thresholds=SKEW_THRESHOLDS, target_field=TARGET
)

### Assemble the objective specification

Finally, you assemble the objective specification (`objective_config`) with the following settings:

- `skew_detection_config`: (Optional) The specification for the skew detection configuration.
- `drift_detection_config`: (Optional) The specification for the drift detection configuration.


In [ ]:
objective_config = model_monitoring.ObjectiveConfig(
    skew_detection_config=skew_config,
    drift_detection_config=drift_config,
)

### Create the input schema

The monitoring service needs to know the features and data types for the the feature inputs to the model, which is referred to as the *input schema*. 

For AutoML models, the *input schema* is predefined and automatically loaded by the monitoring service.

### Create the monitoring job

You create a monitoring job, with your monitoring specifications, using the `aiplatform.ModelDeploymentMonitoringJob.create()` method, with the following parameters:

- `display_name`: The human readable name for the monitoring job.
- `project`: The project ID.
- `location`: The location.
- `endpoint`: The fully qualified resource name of the Vertex AI endpoint to enable monitoring.
- `logging_sampling_strategy`: The specification for the sampling configuration.
- `schedule_config`: The specification for the scheduling configuration.
- `alert_config`: The specification for the alerting configuration.
- `objective_configs`: The specification for the objectives configuration.

In [ ]:
monitoring_job = aiplatform.ModelDeploymentMonitoringJob.create(
    display_name="churn",
    project=PROJECT_ID,
    location=LOCATION,
    endpoint=endpoint,
    logging_sampling_strategy=logging_sampling_strategy,
    schedule_config=schedule_config,
    alert_config=alerting_config,
    objective_configs=objective_config,
)

print(monitoring_job)

#### Email notification of the monitoring job.

An email notification is sent to the email address in the alerting configuration, notifying that the model monitoring job is now enabled.

The contents of the email appear as below:

<blockquote>
Hello Vertex AI Customer,

You are receiving this mail because you are using the Vertex AI Model Monitoring service.
This mail is to inform you that we received your request to set up drift or skew detection for the prediction endpoint listed below. Starting from now, incoming prediction requests will be sampled and logged for analysis.
Raw requests and responses will be collected from prediction service and saved in bq://[your-project-id].model_deployment_monitoring_[endpoint-id].serving_predict .
</blockquote>

*Note:* You do not need to wait for the email notification to continue to the next step.

#### Monitoring Job State

After you start the Vertex AI Model Monitoring job, it stays in a **PENDING** state until `skew distribution baseline` is calculated. The monitoring service initiates a batch job to generate the distribution baseline from the training data. 

Once the baseline distribution is generated, then the monitoring job changes to **OFFLINE** state. On a per interval basis, for example, once an hour, the monitoring job enters **RUNNING** state while analyzing the sampled data. Once completed, it returns to the **OFFLINE** state while awaiting the next scheduled analysis.

In [ ]:
jobs = monitoring_job.list(filter="display_name=churn")
job = jobs[0]
print(job.state)

### Automatic generation of the baseline distribution

Next, the monitoring service creates a batch job to analyze the training data to generate the baseline distribution. Once completed, the monitoring service starts monitoring on the specified intervals.

In [ ]:
import time

# Pause a bit for the baseline distribution to be calculated
if os.getenv("IS_TESTING"):
    time.sleep(180)

### Generate synthetic prediction requests for skew detection

Now, you extract the first 1000 instances from the BigQuery training table for creating prediction requests. Then, modify the data (synthetic) to trigger the skewness detection in the prediction requests by making the follwing updates:

- `year`: Set all values to 3.

In [ ]:
# Download the table.
table = bigquery.TableReference.from_string(DATASET_BQ_URI[5:])

rows = bqclient.list_rows(table, max_results=1000)

instances = []
for row in rows:
    instance = {}
    for key, value in row.items():
        if key == TARGET:
            continue
        if value is None:
            value = ""
        if key == "year":
            value = "3"
        instance[key] = str(value)
    instances.append(instance)

print(len(instances))

### Make the prediction requests

Next, you send the the 1000 prediction requests to your Vertex AI endpoint resource using the `predict()` method.

In [ ]:
responses = []
for i, instance in enumerate(instances):
    response = endpoint.predict(instances=[instance])
    responses.append(response)
    if i % 100 == 0:
        print(f"Completed {i} rows.")

# print the prediction for the first instance
print(responses[0])

### Logging sampled requests

Once the monitoring service has started, the sampled prediction requests are logged to Cloud Storage. On the next monitoring interval, the sampled predictions are copied to the BigQuery logging table. Once the entries are logged, the monitoring service analyzes the sampled data.

Next, you wait for the first logged entries to appear in the BigQuery logging table for prediction samples. Since you sent 1000 prediction requests, with 50% sampling, you should see around 500 entries.

In [ ]:
while True:
    time.sleep(180)

    ENDPOINT_ID = endpoint.resource_name.split("/")[-1]

    table = bigquery.TableReference.from_string(
        f"{PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}.serving_predict"
    )
    rows = bqclient.list_rows(table)
    print(rows.total_rows)
    if rows.total_rows > 0:
        break

### Skew detection during monitoring

The skewness detection for feature inputs occurs at the next monitoring interval. In this tutorial, you set the monitoring interval to one hour. So, in about an hour your monitoring job goes from **OFFLINE** to **RUNNING**. While running, it analyzes the logged sampled tables from the predictions during this interval and compares them to the baseline distribution.

Once the analysis is completed, the monitoring job sends email notifications on the detected skewness, in this case `year`. Further, the monitoring job goes into **OFFLINE** state until the next interval.

#### Wait for monitoring interval 

It can take up to 40 minutes or more from the moment the analysis on the monitoring interval is done until you receive an email alert.

The contents of the email appear as below:

<blockquote>
   Hello Vertex AI Customer,

You are receiving this mail because you are subscribing to the Vertex AI Model Monitoring service.
This mail is just to inform you that there are some anomalies detected in your deployed models and may need your attention.


Basic Information:

Endpoint Name: projects/[your-project-id]/locations/us-central1/endpoints/3315907167046860800
Monitoring Job: projects/[your-project-id]/locations/us-central1/modelDeploymentMonitoringJobs/8672170640054157312
Statistics and Anomalies Root Path(Google Cloud Storage): gs://cloud-ai-platform-773884b1-2a32-48d6-8b83-c03cde416b68/model_monitoring/job-8672170640054157312
BigQuery Command: SELECT * FROM `bq://[your-project-id].model_deployment_monitoring_3315907167046860800.serving_predict`


Training Prediction Skew Anomalies (Raw Feature):

Anomalies Report Path(Google Cloud Storage): gs://cloud-ai-platform-773884b1-2a32-48d6-8b83-c03cde416b68/model_monitoring/job-8672170640054157312/serving/2022-08-25T00:00/stats_and_anomalies/<deployed-model-id>/anomalies/training_prediction_skew_anomalies

For more information about the alert, please visit the model monitoring alert page.

Deployed model id: <deployed-model-id>

Feature name	Anomaly short description	Anomaly long description
country	High Linfty distance between training and serving	The Linfty distance between training and serving is 0.947563 (up to six significant digits), above the threshold 0.5. The feature value with maximum difference is: Year
<blockquote>

In [ ]:
if os.getenv("IS_TESTING"):
    time.sleep(60 * 45)

### Generate synthetic prediction requests for drift detection

Next, you extract the same first 1000 instances from the BigQuery training table to use for prediction requests. Then, modify the data (synthetic) to trigger the drift detection in the prediction requests by making the following updates:

- `month`: set all values to 1.

In [ ]:
# Download the table.
table = bigquery.TableReference.from_string(DATASET_BQ_URI[5:])

rows = bqclient.list_rows(table, max_results=1000)

instances = []
for row in rows:
    instance = {}
    for key, value in row.items():
        if key == TARGET:
            continue
        if value is None:
            value = ""
        elif key == "month":
            value = "1"
        instance[key] = str(value)
    instances.append(instance)

print(len(instances))

### Make the prediction requests

Next, you send the the 1000 prediction requests to your Vertex AI endpoint resource using the `predict()` method.

In [ ]:
responses = []
for i, instance in enumerate(instances):
    response = endpoint.predict(instances=[instance])
    responses.append(response)
    if i % 100 == 0:
        print(f"Completed {i} rows.")

# print the prediction for the first instance
print(responses[0])

### Logging sampled requests

On the next monitoring interval, the sampled predictions are copied to the BigQuery logging table. Once the entries are logged, the monitoring service analyzes the sampled data.

Next, you wait for the first logged entries to appear in the BigQuery logging table for prediction samples. Since you sent 1000 prediction requests, with 50% sampling, you should see around 500 entries.

In [ ]:
while True:
    time.sleep(180)

    ENDPOINT_ID = endpoint.resource_name.split("/")[-1]

    table = bigquery.TableReference.from_string(
        f"{PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}.serving_predict"
    )
    rows = bqclient.list_rows(table)
    print(rows.total_rows)
    if rows.total_rows > 505:
        break

### Drift detection during monitoring

The feature input drift detection occurs at the next monitoring interval. In this tutorial, you set the monitoring interval to one hour. So, in about an hour your monitoring job goes from **OFFLINE** to **RUNNING**. While running, it analyzes the logged sampled tables from the predictions during this interval and compares them to the previous monitoring interval distribution.

Once the analysis is completed, the monitoring job sends email notifications on the detected drift, in this case `cnt_user_engagement`, and the monitoring job goes into **OFFLINE** state until the next interval.

#### Wait for monitoring interval

It can take up to 40 minutes or more from the moment the analyis on the monitoring interval is done until you receive an email alert.

In [ ]:
if os.getenv("IS_TESTING"):
    time.sleep(60 * 45)

### Delete the monitoring job

Once you've received the email alerts and verified the content, you can:
- pause the monitoring job using the `pause()` method.
- delete the monitoring job using the `delete()` method.

In [ ]:
monitoring_job.pause()
monitoring_job.delete()

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

In [ ]:
# Undeploy the model from endpoint before deletion
endpoint.undeploy_all()

# Delete the endpoint
endpoint.delete()

# Delete the model
model.delete()

# Delete the BigQuery table
! bq rm -f {PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}